# Loading libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, concatenate, Flatten, Embedding, TimeDistributed, RepeatVector
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns

# Generating and splitting data for models

In [ ]:
import pandas as pd
merged_data_short = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/merged_data_short.csv")



#CNN-LSTM MODEL WITH SPLIT

## Removing 50 sampled satation

In [ ]:
# Define aggregation dictionary for variables other than 'start_count'
agg_funcs = {col: 'first' for col in merged_data_short.columns if col not in ['station_id_encoded', 'hour', 'date', 'start_count']}
agg_funcs['start_count'] = 'sum'  # Summing up the 'start_count'

# Grouping the data
merged_data_short = merged_data_short.groupby(['station_id_encoded', 'hour', 'date'], as_index=False).agg(agg_funcs)

In [ ]:
# Sampling 50 unique station IDs at random
random_stations = aggregated_df['station_id_encoded'].drop_duplicates().sample(n=50, random_state=42)

# Exclude these stations from the dataset
aggregated_df_sample = aggregated_df[~aggregated_df['station_id_encoded'].isin(random_stations)]


# Generating generic model first

In [ ]:
aggregated_df = merged_data_short[["start_count","date", "station_id_encoded", "hour",
    "temperature_2m (°C)",
    "relativehumidity_2m (%)",
    "precipitation (mm)",
    "snowfall (cm)",
    "cloudcover (%)",
    "IsHoliday",
    "direct_radiation (W/m²)",
    "windspeed_10m (km/h)",
    "year_2021",
    "year_2022",
    "year_2023",
    "month_name_April",
    "month_name_August",
    "month_name_December",
    "month_name_February",
    "month_name_January",
    "month_name_July",
    "month_name_June",
    "month_name_March",
    "month_name_May",
    "month_name_November",
    "month_name_October",
    "month_name_September",
    "day_of_week_Friday",
    "day_of_week_Monday",
    "day_of_week_Saturday",
    "day_of_week_Sunday",
    "day_of_week_Thursday",
    "day_of_week_Tuesday",
    "day_of_week_Wednesday"]]


In [ ]:
aggregated_df_sample['date'] = pd.to_datetime(aggregated_df_sample['date'])
def time_of_day_dummy(hour):
    if 0 <= hour < 7:
        return 'early_morning'
    elif 7 <= hour < 9:
        return 'morning_rush'
    elif 9 <= hour < 12:
        return 'late_morning'
    elif 12 <= hour < 16:
        return 'lunch_time'
    elif 16 <= hour < 19:
        return 'afternoon_rush'
    elif 19 <= hour <= 23:
        return 'evening'
    else:
        return 'invalid_hour'  # Just in case there are any unexpected hour values

# Apply the function to create a new column with time of day categories
aggregated_df_sample['time_of_day'] = aggregated_df_sample['hour'].apply(time_of_day_dummy)

# Now we create dummy variables from the 'time_of_day' column
time_of_day_dummies = pd.get_dummies(aggregated_df_sample['time_of_day'], prefix='dummy')

# Join the dummy variables with the main dataframe
aggregated_df_sample = aggregated_df_sample.join(time_of_day_dummies)

# Drop the 'time_of_day' column as it's no longer needed
aggregated_df_sample.drop('time_of_day', axis=1, inplace=True)

# Show the dataframe to confirm the changes
aggregated_df_sample.head()



In [ ]:
def is_weekend(date):
    if date.weekday() >= 5:
        return 1
    else:
        return 0

# Apply the function to create the weekend dummy variable
aggregated_df_sample['weekend_dummy'] = aggregated_df_sample['date'].apply(is_weekend)


## Splitting and standardizing data

In [ ]:
# Sort by 'date' (and other relevant columns if necessary)
merged_data_short_sorted = aggregated_df_sample.sort_values(by=['date', 'station_id_encoded'])
# Reset index after sorting
merged_data_short_sorted.reset_index(drop=True, inplace=True)

# Split sorted data into features (X) and target (y)
X_sorted = merged_data_short_sorted.drop('start_count', axis=1)
y_sorted = merged_data_short_sorted['start_count']

# Splitting data on date
train_end_index = X_sorted[X_sorted['date'] == '2022-11-30'].index[-1]
val_end_index = X_sorted[X_sorted['date'] == '2023-03-10'].index[-1]

# Split the data manually
X_train = X_sorted.iloc[:train_end_index]
y_train = y_sorted.iloc[:train_end_index]
X_val = X_sorted.iloc[train_end_index:val_end_index]
y_val = y_sorted.iloc[train_end_index:val_end_index]
X_test = X_sorted.iloc[val_end_index:]
y_test = y_sorted.iloc[val_end_index:]


In [ ]:
X_train.drop(columns=["date"], inplace=True)
X_val.drop(columns=["date"], inplace=True)
X_test.drop(columns=["date"], inplace=True)


In [ ]:
# Feature columns to be standardized
feature_columns = ["hour", "temperature_2m (°C)",
                   "relativehumidity_2m (%)", "precipitation (mm)", "snowfall (cm)",
                   "cloudcover (%)", "direct_radiation (W/m²)", "windspeed_10m (km/h)"]

# Create the Normalization layer and adapt it to the training data
standardizer = tf.keras.layers.Normalization(axis=-1)
standardizer.adapt(X_train[feature_columns])

# Apply the standardizer to the training data
X_train[feature_columns] = standardizer(X_train[feature_columns].values)

# Apply the standardizer to the validation data
X_val[feature_columns] = standardizer(X_val[feature_columns].values)

# Apply the standardizer to the test data
X_test[feature_columns] = standardizer(X_test[feature_columns].values)


# Adapt the standardizer to the training data target variable
target_standardizer = tf.keras.layers.Normalization(axis=-1)

target_standardizer.adapt(y_train.to_frame())

# Apply the standardizer to the training data target variable
y_train = target_standardizer(y_train.to_frame()).numpy().flatten()

# Apply the standardizer to the validation data target variable
y_val = target_standardizer(y_val.to_frame()).numpy().flatten()

# Apply the standardizer to the test data target variable
y_test = target_standardizer(y_test.to_frame()).numpy().flatten()


In [ ]:
X_train.drop(columns=["station_id_encoded"], inplace=True)
X_val.drop(columns=["station_id_encoded"], inplace=True)
X_test.drop(columns=["station_id_encoded"], inplace=True)

In [ ]:
y_train = pd.DataFrame(y_train, columns=['start_count'])
y_test = pd.DataFrame(y_test, columns=['start_count'])
y_val = pd.DataFrame(y_val, columns=['start_count'])

## Modified batch generator


In [ ]:
def modified_batch_generator_without_station_ids(X, y, time_steps=24, batch_size=256, infinite_loop=True):
    total_size = len(X) - time_steps
    start_idx = 0

    while True:
        X_batch = np.zeros((batch_size, time_steps, X.shape[1]))
        y_batch = np.zeros((batch_size, time_steps, 1))

        for i in range(batch_size):
            if start_idx + time_steps <= total_size:
                X_batch[i] = X.iloc[start_idx:start_idx + time_steps].values
                y_batch[i] = y.iloc[start_idx:start_idx + time_steps].values.reshape(-1, 1)
                start_idx += 1
            else:
                if infinite_loop:
                    start_idx = 0
                else:
                    break

        yield (X_batch, y_batch)

        if not infinite_loop and start_idx + time_steps > total_size:
            break

# Example usage
train_gen_without_station_ids = modified_batch_generator_without_station_ids(X_train, y_train, time_steps=24, batch_size=256, infinite_loop=True)
val_gen_without_station_ids = modified_batch_generator_without_station_ids(X_val, y_val, time_steps=24, batch_size=256, infinite_loop=True)
test_gen_without_station_ids = modified_batch_generator_without_station_ids(X_test, y_test, time_steps=24, batch_size=256, infinite_loop=False)

# Get the next batch from the generator
batch_output = next(train_gen_without_station_ids)

# Extract X_batch and y_batch
X_batch, y_batch = batch_output

# Print shapes
print("X Batch Shape:", X_batch.shape)
print("y Batch Shape:", y_batch.shape)


## Generic feature model

In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

keras.backend.clear_session()
def build_generic_feature_model(input_shape):
    # Input for generic features
    input_shape = (24, X_batch.shape[2])

    generic_input = Input(shape=input_shape)

    # CNN-LSTM architecture
    conv_layer = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(generic_input)
    lstm_layer = LSTM(units=70, return_sequences=True, activation='relu')(conv_layer)
    lstm_layer_3 = Dropout(rate=0.4)(lstm_layer)
    # Dense layers for prediction
    time_distributed= TimeDistributed(Dense(20))(lstm_layer_3)
    time_distributed_output = TimeDistributed(Dense(1))(time_distributed)

    model = Model(inputs=generic_input, outputs=time_distributed_output)
    model.compile(optimizer=RMSprop(learning_rate=0.001), loss='mse', metrics=['mae', rmse])
    model.summary()
    return model

generic_feature_model = build_generic_feature_model(input_shape=(24, X_batch.shape[2]))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24, 38)]          0         
                                                                 
 conv1d (Conv1D)             (None, 24, 64)            12224     
                                                                 
 lstm (LSTM)                 (None, 24, 70)            37800     
                                                                 
 dropout (Dropout)           (None, 24, 70)            0         
                                                                 
 time_distributed (TimeDist  (None, 24, 20)            1420      
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 24, 1)             21        
 stributed)                                                  

# Running generic model

In [ ]:
# Callback for early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
import os
from tensorflow.keras.models import load_model


checkpoint_dir = '/content/drive/MyDrive/Colab Notebooks/checkpoint_directory'
list_of_files = os.listdir(checkpoint_dir)
# Filter out files that match your checkpoint pattern and sort them
checkpoint_files = sorted([f for f in list_of_files if 'model_cnn-lstm-generic' in f])

if checkpoint_files:
    last_checkpoint = checkpoint_files[-1]
    last_model_path = os.path.join(checkpoint_dir, last_checkpoint)
    # Load the last model with the custom rmse function
    generic_feature_model = load_model(last_model_path, custom_objects={'rmse': rmse})
else:
    print("No checkpoint found. Please train the model from scratch.")


In [ ]:
# Configure ModelCheckpoint

from tensorflow.keras.callbacks import ModelCheckpoint
# Define ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Colab Notebooks/checkpoint_directory/model_cnn-lstm-generic{epoch:02d}-{loss:.2f}.h5',
    monitor='loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

steps_per_epoch = len(X_train) // batch_size

validation_steps = len(X_val) // batch_size

# Train the model using the generator
history = generic_feature_model.fit(
    x=train_gen_without_station_ids,
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    validation_data=val_gen_without_station_ids,
    validation_steps=validation_steps,
    callbacks=[early_stopping, checkpoint]
)

# Generating predictions

In [ ]:
from tensorflow.keras.models import load_model

# Replace this path with the path to your model file
model_path ="/content/drive/MyDrive/Colab Notebooks/checkpoint_directory/model_cnn-lstm-generic09-0.61.h5"

# Load the model
generic_feature_model_1 = load_model(model_path, custom_objects={'rmse': rmse})


In [ ]:
import numpy as np

def reshape_and_pad_data(X, y, time_steps=24):
    # Calculate the padding size
    pad_size = time_steps - (X.shape[0] % time_steps)
    pad_size = pad_size if pad_size != time_steps else 0

    # Pad the data
    X_padded = np.vstack([X, X.iloc[:pad_size]])
    y_padded = np.vstack([y, y.iloc[:pad_size]])

    # Reshape the data
    X_reshaped = X_padded.reshape((-1, time_steps, X.shape[1]))
    y_reshaped = y_padded.reshape((-1, time_steps, 1))  # Add an extra dimension for y

    return X_reshaped, y_reshaped

# Reshape and pad test data
X_reshaped, y_reshaped = reshape_and_pad_data(X_test, y_test)


In [ ]:
# Now station_id_batch_reshaped should be ready for use in prediction
test_predictions_batch = generic_feature_model_1.predict(X_reshaped)


1060/1060 [==============================] - 7s 6ms/step


In [ ]:
# Flatten the last two dimensions for both predictions and actuals
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
y_test_flat = y_reshaped.reshape(-1)  # Reshape to (51240 * 24,)
test_predictions_flat = test_predictions_batch.reshape(-1)  # Reshape to (51240 * 24,)

# Calculate MAE and RMSE
test_mae = mean_absolute_error(y_test_flat, test_predictions_flat)
test_rmse = sqrt(mean_squared_error(y_test_flat, test_predictions_flat))

print('Test MAE:', test_mae)
print('Test RMSE:', test_rmse)


# Exporting predictions

In [ ]:
def unpad_predictions(predictions, original_data, time_steps=24):
    # Calculate the pad size used during padding
    pad_size = time_steps - (original_data.shape[0] % time_steps)
    pad_size = pad_size if pad_size != time_steps else 0

    # Remove the padding
    predictions_unpadded = predictions[:-pad_size] if pad_size != 0 else predictions

    return predictions_unpadded

# Unpad the predictions
unpadded_predictions = unpad_predictions(test_predictions_flat, y_test)


In [ ]:
# Convert the numpy array to a pandas DataFrame
unpadded_predictions_df = pd.DataFrame(unpadded_predictions, columns=['prediction'])

# Export to a CSV file
unpadded_predictions_df.to_csv('/content/drive/MyDrive/Colab Notebooks/general_standardized_predictions_df_normal9.csv', index=False)


# Exporting unstandardized predictions

In [ ]:
# Retrieve the mean and variance from the target_standardizer
mean, variance = target_standardizer.mean.numpy(), target_standardizer.variance.numpy()

# Compute the standard deviation from the variance
std_dev = np.sqrt(variance)

# Extract single value for mean and standard deviation
mean_value = mean.item()  # Assuming mean is a numpy array with a single element
std_dev_value = np.sqrt(variance).item()  # Assuming variance is a numpy array with a single element

# Apply the inverse transformation formula to unstandardize adjusted_test_predictions_flat
unstandardized_test_predictions_flat = unpadded_predictions * std_dev_value + mean_value


In [ ]:
import pandas as pd

# Convert the numpy array to a pandas DataFrame
predictions_df = pd.DataFrame(unstandardized_test_predictions_flat, columns=['prediction'])

# Export to a CSV file
predictions_df.to_csv('/content/drive/MyDrive/Colab Notebooks/general_unstandardized_predictions.csv', index=False)


# Getting predictions for full dataset to substract from specific model

In [ ]:
import pandas as pd
merged_data_short = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/merged_data_short.csv")

In [ ]:
# Define aggregation dictionary for variables other than 'start_count'
agg_funcs = {col: 'first' for col in merged_data_short.columns if col not in ['station_id_encoded', 'hour', 'date', 'start_count']}
agg_funcs['start_count'] = 'sum'
# Grouping the data
merged_data_short = merged_data_short.groupby(['station_id_encoded', 'hour', 'date'], as_index=False).agg(agg_funcs)

In [ ]:
aggregated_df = merged_data_short[["start_count","date", "station_id_encoded", "hour",
    "temperature_2m (°C)",
    "relativehumidity_2m (%)",
    "precipitation (mm)",
    "snowfall (cm)",
    "cloudcover (%)",
    "IsHoliday",
    "direct_radiation (W/m²)",
    "windspeed_0m (km/h)",
    "year_2021",
    "year_2022",
    "year_2023",
    "month_name_April",
    "month_name_August",
    "month_name_December",
    "month_name_February",
    "month_name_January",
    "month_name_July",
    "month_name_June",
    "month_name_March",
    "month_name_May",
    "month_name_November",
    "month_name_October",
    "month_name_September",
    "day_of_week_Friday",
    "day_of_week_Monday",
    "day_of_week_Saturday",
    "day_of_week_Sunday",
    "day_of_week_Thursday",
    "day_of_week_Tuesday",
    "day_of_week_Wednesday"]]

In [ ]:
# Sampling 50 unique station IDs at random
random_stations = aggregated_df['station_id_encoded'].drop_duplicates().sample(n=50, random_state=42)

# Exclude these stations from the dataset
aggregated_df_sample = aggregated_df[~aggregated_df['station_id_encoded'].isin(random_stations)]


In [ ]:
# Sort by 'date' (and other relevant columns if necessary)
merged_data_short_sorted = aggregated_df_sample.sort_values(by=['date', 'station_id_encoded'])
# Reset index after sorting
merged_data_short_sorted.reset_index(drop=True, inplace=True)


In [ ]:
# Split sorted data into features (X) and target (y)
X_sorted = merged_data_short_sorted.drop('start_count', axis=1)
y_sorted = merged_data_short_sorted['start_count']

In [ ]:
import tensorflow as tf
# Feature columns to be standardized
feature_columns = ["hour", "temperature_2m (°C)",
                   "relativehumidity_2m (%)", "precipitation (mm)", "snowfall (cm)",
                   "cloudcover (%)", "direct_radiation (W/m²)", "windspeed_10m (km/h)"]

# Create the Normalization layer and adapt it to the training data
standardizer = tf.keras.layers.Normalization(axis=-1)
standardizer.adapt(X_sorted[feature_columns])

# Apply the standardizer to the training data
X_sorted[feature_columns] = standardizer(X_sorted[feature_columns].values)

In [ ]:
# Adapt the standardizer to the training data target variable
target_standardizer = tf.keras.layers.Normalization(axis=-1)

target_standardizer.adapt(y_sorted.to_frame())

# Apply the standardizer to the training data target variable
y_sorted = target_standardizer(y_sorted.to_frame()).numpy().flatten()


In [ ]:
X_sorted.drop(columns=["station_id_encoded"], inplace=True)
X_sorted.drop(columns=["date"], inplace=True)
y_sorted = pd.DataFrame(y_sorted, columns=['start_count'])

In [ ]:
from tensorflow.keras.models import load_model

# Replace this path with the path to your model file
model_path = "/content/drive/MyDrive/Colab Notebooks/checkpoint_directory/model_cnn-lstm-generic09-0.63.h5"

# Load the model
generic_feature_model_1 = load_model(model_path, custom_objects={'rmse': rmse})


In [ ]:
import numpy as np

def reshape_and_pad_data(X, y, time_steps=24):
    # Calculate the padding size
    pad_size = time_steps - (X.shape[0] % time_steps)
    pad_size = pad_size if pad_size != time_steps else 0

    # Pad the data
    X_padded = np.vstack([X, X.iloc[:pad_size]])
    y_padded = np.vstack([y, y.iloc[:pad_size]])

    # Reshape the data
    X_reshaped = X_padded.reshape((-1, time_steps, X.shape[1]))
    y_reshaped = y_padded.reshape((-1, time_steps, 1))  # Add an extra dimension for y

    return X_reshaped, y_reshaped

# Reshape and pad test data
X_reshaped, y_reshaped = reshape_and_pad_data(X_sorted, y_sorted)


# Predict using full dataset

In [ ]:
# Now station_id_batch_reshaped should be ready for use in prediction
test_predictions_batch = generic_feature_model_1.predict(X_reshaped)


4936/4936 [==============================] - 26s 5ms/step


In [ ]:
# Flatten the last two dimensions for both predictions and actuals
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
y_train_flat = y_reshaped.reshape(-1)  # Reshape to (51240 * 24,)
train_predictions_flat = test_predictions_batch.reshape(-1)  # Reshape to (51240 * 24,)

# Calculate MAE and RMSE
test_mae = mean_absolute_error(y_train_flat, train_predictions_flat)
test_rmse = sqrt(mean_squared_error(y_train_flat, train_predictions_flat))

print('Test MAE:', test_mae)
print('Test RMSE:', test_rmse)


# Export predictions for later use

In [ ]:
def unpad_predictions(predictions, original_data, time_steps=24):
    # Calculate the pad size used during padding
    pad_size = time_steps - (original_data.shape[0] % time_steps)
    pad_size = pad_size if pad_size != time_steps else 0

    # Remove the padding
    predictions_unpadded = predictions[:-pad_size] if pad_size != 0 else predictions

    return predictions_unpadded

# Unpad the predictions
unpadded_predictions = unpad_predictions(train_predictions_flat, y_sorted)


In [ ]:
import pandas as pd

# Convert the numpy array to a pandas DataFrame
unpadded_predictions_ = pd.DataFrame(unpadded_predictions, columns=['prediction'])

# Export to a CSV file
unpadded_predictions_.to_csv('/content/drive/MyDrive/Colab Notebooks/predictions_full_generic_model_standardized_new.csv', index=False)


## Export unstandardized predictions

In [ ]:
# Retrieve the mean and variance from the target_standardizer
mean, variance = target_standardizer.mean.numpy(), target_standardizer.variance.numpy()

# Compute the standard deviation from the variance
std_dev = np.sqrt(variance)

# Extract single value for mean and standard deviation
mean_value = mean.item()  # Assuming mean is a numpy array with a single element
std_dev_value = np.sqrt(variance).item()  # Assuming variance is a numpy array with a single element

# Apply the inverse transformation formula to unstandardize adjusted_train_predictions_flat
unstandardized_train_predictions_flat = unpadded_predictions * std_dev_value + mean_value


In [ ]:
# Convert the numpy array to a pandas DataFrame
predictions_full_generic_model_train = pd.DataFrame(unstandardized_train_predictions_flat, columns=['prediction'])

# Export to a CSV file
predictions_full_generic_model_train.to_csv('/content/drive/MyDrive/Colab Notebooks/predictions_full_generic_model_unstandardized_new.csv', index=False)


# Specific model

In [ ]:
predictions_full_generic_model_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predictions_full_generic_model_train.csv")

In [ ]:
merged_data_short = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/merged_data_short.csv")

In [ ]:
# Define aggregation dictionary for variables other than 'start_count'
agg_funcs = {col: 'first' for col in merged_data_short.columns if col not in ['station_id_encoded', 'hour', 'date', 'start_count']}
agg_funcs['start_count'] = 'sum'

# Grouping the data
merged_data_short = merged_data_short.groupby(['station_id_encoded', 'hour', 'date'], as_index=False).agg(agg_funcs)

In [ ]:
merged_data_short_specific = merged_data_short[["station_id_encoded", "start_count", "latitude",
                                                "longitude", "bike_lane_length_km",
                                                "restaurants_count", "rail_stations_count",
                                                "universities_count", "bus_stations_count",
                                                "parks_count", "date", "hour"]]

In [ ]:
# Sampling 50 unique station IDs at random
random_stations = merged_data_short_specific['station_id_encoded'].drop_duplicates().sample(n=50, random_state=42)

# Exclude these stations from the dataset
merged_data_short_specific = merged_data_short_specific[~merged_data_short_specific['station_id_encoded'].isin(random_stations)]


## Splitting and standardizing data

In [ ]:

merged_data_short_sorted = merged_data_short_specific.sort_values(by=['date', 'station_id_encoded'])
# Reset index after sorting
merged_data_short_sorted.reset_index(drop=True, inplace=True)

In [ ]:
# Split sorted data into features (X) and target (y)
X_sorted = merged_data_short_sorted.drop('start_count', axis=1)
y_sorted = merged_data_short_sorted['start_count']

# Split on date
train_end_index = X_sorted[X_sorted['date'] == '2022-11-30'].index[-1]
val_end_index = X_sorted[X_sorted['date'] == '2023-03-10'].index[-1]

# Split the data manually
X_train = X_sorted.iloc[:train_end_index]
y_train = y_sorted.iloc[:train_end_index]
X_val = X_sorted.iloc[train_end_index:val_end_index]
y_val = y_sorted.iloc[train_end_index:val_end_index]
X_test = X_sorted.iloc[val_end_index:]
y_test = y_sorted.iloc[val_end_index:]

In [ ]:
y_train = pd.DataFrame(y_train, columns=['start_count'])
y_test = pd.DataFrame(y_test, columns=['start_count'])
y_val = pd.DataFrame(y_val, columns=['start_count'])

In [ ]:
# Reference the predictions column
predictions_column = predictions_full_generic_model_train['prediction']
test = predictions_column.iloc[:train_end_index]

### Substract predictions from train data for specific model

In [ ]:
# Ensure the lengths match
if len(y_train) == len(test):
    # Perform the subtraction
    y_train['start_count'] = y_train['start_count'] - test
else:
    print("Error: The lengths of the DataFrame and the predictions array do not match.")

In [ ]:
X_train.drop(columns=["date"], inplace=True)
X_val.drop(columns=["date"], inplace=True)
X_test.drop(columns=["date"], inplace=True)

In [ ]:
import tensorflow as tf
feature_columns = ["latitude", "longitude",
                   "bike_lane_length_km", "restaurants_count", "rail_stations_count",
                   "universities_count", "bus_stations_count", "parks_count"]


# Create the Normalization layer and adapt it to the training data
standardizer = tf.keras.layers.Normalization(axis=-1)
standardizer.adapt(X_train[feature_columns])

# Apply the standardizer to the training data
X_train[feature_columns] = standardizer(X_train[feature_columns].values)

# Apply the standardizer to the validation data
X_val[feature_columns] = standardizer(X_val[feature_columns].values)

# Apply the standardizer to the test data
X_test[feature_columns] = standardizer(X_test[feature_columns].values)

# Adapt the standardizer to the training data target variable
target_standardizer = tf.keras.layers.Normalization(axis=-1)

target_standardizer.adapt(y_train)

# Apply the standardizer to the training data target variable
y_train = target_standardizer(y_train).numpy().flatten()

# Apply the standardizer to the validation data target variable
y_val = target_standardizer(y_val).numpy().flatten()

# Apply the standardizer to the test data target variable
y_test = target_standardizer(y_test).numpy().flatten()

In [ ]:
y_train = pd.DataFrame(y_train, columns=['start_count'])
y_test = pd.DataFrame(y_test, columns=['start_count'])
y_val = pd.DataFrame(y_val, columns=['start_count'])

X_train.drop(columns=["hour"], inplace=True)
X_val.drop(columns=["hour"], inplace=True)
X_test.drop(columns=["hour"], inplace=True)

In [ ]:
# Number of unique stations in the dataset
num_stations = X_val['station_id_encoded'].nunique()

# Maximum value of station ID (needed to define the input dimension of the embedding layer)
max_station_id = int(X_val['station_id_encoded'].max()) + 1

# Define the embedding size
# Rule of thumb: min(50, number of categories/2)
embedding_size = min(50, num_stations // 2)
num_stations

In [ ]:
station_ids_train = X_train['station_id_encoded']
station_ids_val = X_val['station_id_encoded']
station_ids_test = X_test['station_id_encoded']

# Batch generator

In [ ]:
import numpy as np

def modified_batch_generator(X, y, station_ids, time_steps=24, batch_size=256, infinite_loop=True):
    total_size = len(X)
    start_idx = 0

    while True:
        X_batch = np.zeros((batch_size, time_steps, X.shape[1]))
        station_id_batch = np.zeros((batch_size, 1))
        y_batch = np.zeros((batch_size, time_steps, 1))

        for i in range(batch_size):
            if start_idx + time_steps <= total_size:
                X_seq = X.iloc[start_idx:start_idx + time_steps]
                y_seq = y.iloc[start_idx:start_idx + time_steps]
                station_id = station_ids.iloc[start_idx + time_steps - 1]

                # Padding in case of insufficient data
                X_pad = np.zeros((time_steps, X.shape[1]))
                y_pad = np.zeros((time_steps, 1))
                X_pad[:len(X_seq)] = X_seq.values
                y_pad[:len(y_seq)] = y_seq.values.reshape(-1, 1)

                X_batch[i] = X_pad
                station_id_batch[i] = station_id
                y_batch[i] = y_pad
                start_idx += 1
            else:
                if infinite_loop:
                    start_idx = 0
                else:
                    break

        yield ([X_batch, station_id_batch], y_batch)

        if not infinite_loop and start_idx + time_steps > total_size:
            break

# Recreate generators with the modified batch generator
train_gen = modified_batch_generator(X_train, y_train, station_ids_train, time_steps=24, batch_size=256, infinite_loop=True)
val_gen = modified_batch_generator(X_val, y_val, station_ids_val, time_steps=24, batch_size=256, infinite_loop=True)
test_gen = modified_batch_generator(X_test, y_test, station_ids_test, time_steps=24, batch_size=256, infinite_loop=False)

batch_size = 256
# Get the next batch from the generator
batch_output = next(train_gen)

# Extract station_id_batch, X_batch, and y_batch
X_batch, station_id_batch = batch_output[0]
y_batch = batch_output[1]

# Print shapes
print("Station ID Batch Shape:", station_id_batch.shape)
print("X Batch Shape:", X_batch.shape)
print("y Batch Shape:", y_batch.shape)

## Specific model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, concatenate, Conv1D, Flatten, Embedding, TimeDistributed, RepeatVector
from tensorflow.keras import regularizers

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

cnn_lstm_input_shape = (24, X_batch.shape[2])
station_id_input_shape = (1,)

def build_cnn_lstm_model_specific(cnn_lstm_input_shape, station_id_input_shape):
    cnn_lstm_input = Input(shape=cnn_lstm_input_shape)
    conv1 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(cnn_lstm_input)
    lstm = Dropout(rate=0.2)(conv1)
    lstm1 = LSTM(units=70, activation='relu', return_sequences=True)(lstm)
    lstm2 = Dropout(rate=0.2)(lstm1)


    station_id_input = Input(shape=station_id_input_shape)
    station_id_embedding = Embedding(input_dim=max_station_id + 1, output_dim=embedding_size)(station_id_input)
    station_id_embedding = Flatten()(station_id_embedding)
    station_id_embedding = RepeatVector(24)(station_id_embedding)  # Repeat to match time steps

    combined = concatenate([lstm2, station_id_embedding], axis=-1)
    time_distributed_output = TimeDistributed(Dense(64))(combined)
    final_output = TimeDistributed(Dense(1))(time_distributed_output)  # Predicting for each time step

    optimizer = RMSprop(learning_rate=0.0001)

    model = Model(inputs=[cnn_lstm_input, station_id_input], outputs=final_output)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae', rmse])

    return model

cnn_lstm_model_specific_no_cap = build_cnn_lstm_model_specific(cnn_lstm_input_shape, station_id_input_shape)
cnn_lstm_model_specific_no_cap.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 24, 10)]             0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 24, 64)               3264      ['input_1[0][0]']             
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 dropout (Dropout)           (None, 24, 64)               0         ['conv1d[0][0]']              
                                                                                              

In [ ]:
# Callback for early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/Colab Notebooks/checkpoint_directory'
list_of_files = os.listdir(checkpoint_dir)
# Filter out files that match your checkpoint pattern and sort them
checkpoint_files = sorted([f for f in list_of_files if 'cnn_lstm_model_specific_no_cap' in f])

if checkpoint_files:
    last_checkpoint = checkpoint_files[-1]
    last_model_path = os.path.join(checkpoint_dir, last_checkpoint)
    # Load the last model with the custom rmse function
    cnn_lstm_model_specific_no_cap = load_model(last_model_path, custom_objects={'rmse': rmse})
else:
    print("No checkpoint found. Please train the model from scratch.")


In [ ]:

checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Colab Notebooks/checkpoint_directory/cnn_lstm_model_specific_no_cap{epoch:02d}-{val_loss:.2f}.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

steps_per_epoch = len(X_train) // batch_size
# Calculate the number of steps in the validation generator
validation_steps = len(X_val) // batch_size

history = cnn_lstm_model_specific_no_cap.fit(
    x=train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    validation_data=val_gen,
    validation_steps=validation_steps,
    callbacks=[early_stopping, checkpoint]
)



## Generating predictions for specific model

In [ ]:
from tensorflow.keras.models import load_model

# Replace this path with the path to your model file
model_path = "/content/drive/MyDrive/Colab Notebooks/checkpoint_directory/cnn_lstm_model_no_cap-2.23.h5"

# Load the model
specific_feature_model = load_model(model_path, custom_objects={'rmse': rmse})


In [ ]:
import numpy as np

def reshape_and_pad_data(X, y, time_steps=24):
    # Calculate the padding size
    pad_size = time_steps - (X.shape[0] % time_steps)
    pad_size = pad_size if pad_size != time_steps else 0

    # Pad the data
    X_padded = np.vstack([X, X.iloc[:pad_size]])
    y_padded = np.vstack([y, y.iloc[:pad_size]])

    # Reshape the data
    X_reshaped = X_padded.reshape((-1, time_steps, X.shape[1]))
    y_reshaped = y_padded.reshape((-1, time_steps, 1))  # Add an extra dimension for y

    return X_reshaped, y_reshaped

# Reshape and pad test data
X_test_reshaped, y_test_reshaped = reshape_and_pad_data(X_test, y_test)


In [ ]:
def replicate_and_reshape_station_ids(station_ids, num_sequences):
    # Calculate the exact number of replications needed
    num_replications = int(np.ceil(num_sequences / len(station_ids)))

    # Replicate the station IDs
    station_ids_replicated = np.repeat(station_ids, num_replications)

    # Trim the replicated array to match the exact number of sequences
    station_ids_trimmed = station_ids_replicated[:num_sequences]

    # Reshape the station IDs to match the required format
    station_ids_reshaped = station_ids_trimmed.reshape((num_sequences, 1))

    return station_ids_reshaped

# Use the function to reshape station_id_batch
num_sequences = X_test_reshaped.shape[0]
station_id_batch_reshaped = replicate_and_reshape_station_ids(station_id_batch, num_sequences)


In [ ]:
# Now station_id_batch_reshaped should be ready for use in prediction
test_predictions_batch = specific_feature_model.predict([X_test_reshaped, station_id_batch_reshaped])


1060/1060 [==============================] - 5s 5ms/step


In [ ]:
# Flatten the last two dimensions for both predictions and actuals
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
y_test_flat = y_test_reshaped.reshape(-1)  # Reshape to (51240 * 24,)
test_predictions_flat = test_predictions_batch.reshape(-1)  # Reshape to (51240 * 24,)

# Calculate MAE and RMSE
test_mae = mean_absolute_error(y_test_flat, test_predictions_flat)
test_rmse = sqrt(mean_squared_error(y_test_flat, test_predictions_flat))

print('Test MAE:', test_mae)
print('Test RMSE:', test_rmse)


# Export predictions

In [ ]:
def unpad_predictions(predictions, original_data, time_steps=24):
    # Calculate the pad size used during padding
    pad_size = time_steps - (original_data.shape[0] % time_steps)
    pad_size = pad_size if pad_size != time_steps else 0

    # Remove the padding
    predictions_unpadded = predictions[:-pad_size] if pad_size != 0 else predictions

    return predictions_unpadded

# Unpad the predictions
unpadded_predictions = unpad_predictions(test_predictions_flat, y_test)


In [ ]:
# Convert the numpy array to a pandas DataFrame
unpadded_predictions_ = pd.DataFrame(unpadded_predictions, columns=['prediction'])

# Export to a CSV file
unpadded_predictions_.to_csv('/content/drive/MyDrive/Colab Notebooks/standardized_predictions_specific_model_cnn_lstm_model_no_cap.csv', index=False)


In [ ]:
# Retrieve the mean and variance from the target_standardizer
mean, variance = target_standardizer.mean.numpy(), target_standardizer.variance.numpy()

# Compute the standard deviation from the variance
std_dev = np.sqrt(variance)

# Extract single value for mean and standard deviation
mean_value = mean.item()  # Assuming mean is a numpy array with a single element
std_dev_value = np.sqrt(variance).item()  # Assuming variance is a numpy array with a single element

# Apply the inverse transformation formula to unstandardize adjusted_train_predictions_flat
unstandardized_test_predictions_flat = unpadded_predictions * std_dev_value + mean_value


In [ ]:
# Convert the numpy array to a pandas DataFrame
unstandardized_test_predictions_flat = pd.DataFrame(unstandardized_test_predictions_flat, columns=['prediction'])
unstandardized_test_predictions_flat.to_csv('/content/drive/MyDrive/Colab Notebooks/standardized_predictions_specific_model_cnn_lstm_model_no_cap.csv', index=False)


# Result for splitted CNN-LSTM model

## Import both predictions

In [ ]:
import pandas as pd
standardized_predictions_specific_model_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/unstandardized_predictions_specific_model_cnn_lstm_model_no_cap.csv")

import pandas as pd
standardized_predictions_general = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/general_standardized_predictions_df_normal8.csv")



## Sum up the predictions

In [ ]:
# Convert DataFrame to a NumPy array and then perform matrix addition
if standardized_predictions_general.shape == standardized_predictions_specific_model_test.shape:
    result = standardized_predictions_general + standardized_predictions_specific_model_test
else:
    print("DataFrame and array do not have the same shape.")

In [ ]:
result2 = result.to_numpy()

## Generate performance

In [ ]:
# Flatten the last two dimensions for both predictions and actuals
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

# Calculate MAE and RMSE
test_mae = mean_absolute_error(y_test, result2)
test_rmse = sqrt(mean_squared_error(y_test, result2))

print('Test MAE:', test_mae)
print('Test RMSE:', test_rmse)
